In [2]:
# !git clone --recursive https://github.com/Microsoft/LightGBM
# %cd /content/LightGBM
# !mkdir build
# !cmake -DUSE_GPU=1 #avoid ..
# !make -j$(nproc)
# !sudo apt-get -y install python-pip
# !sudo -H pip install setuptools pandas numpy scipy scikit-learn -U
# %cd /content/LightGBM/python-package
# !sudo python setup.py install --precompile

%cd /content
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# enter the foldername in your Drive where you have saved the unzipped
# assignment folder, e.g. 'cs231n/assignments/assignment3/'
FOLDERNAME = "mango_class/"
assert FOLDERNAME is not None, "[!] Enter the foldername."

# now that we've mounted your Drive, this ensures that
# the Python interpreter of the Colab VM can load
# python files from within it.
import sys
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))

# this downloads the CIFAR-10 dataset to your Drive
# if it doesn't already exist.
%cd drive/My\ Drive/$FOLDERNAME/

/content
Mounted at /content/drive
/content/drive/My Drive/mango_class


In [2]:
# !rm -rf LightGBM

In [3]:

# !rm -rf LightGBM
import lightgbm as lgb
# %cd ..
# %cd build
# !cmake -DUSE_GPU=1 ..

In [4]:
import os
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.image as mpimg
from PIL import Image#https://yungyuc.github.io/oldtech/python/python_imaging.html
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset, DataLoader
import glob
import numpy as np
from PIL import Image#https://yungyuc.github.io/oldtech/python/python_imaging.html
import torch.optim as optim
from torch.optim import lr_scheduler


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [5]:

import torchvision.models as models
import torch.nn as nn
import time
from py_files.dataloader2 import *
import copy

In [6]:
import torch
USE_GPU = True

# dtype = torch.float32 # we will be using float throughout this tutorial

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# Constant to control how frequently we print train loss
print_every = 100

print('using device:', device)


using device: cuda


In [7]:
X = torch.empty(0,4608).to(device)
y = torch.empty(0, dtype=torch.long).to(device)
for i in range(1,5):
  loal = torch.load(open("./model_params/nobatch" + str(i) + "_feat_extra.pth", 'rb'))
  X = torch.cat((X, loal), dim = 0)

  loal2 = torch.load(open("./model_params/nobatch" + str(i) + "_label.pth", 'rb'))
  y = torch.cat((y, loal2), dim = 0)

X_dev = torch.load(open("./model_params/dev_feat_extra.pth", 'rb'))
y_dev = torch.load(open("./model_params/dev_label.pth", 'rb'))


In [8]:
print(X[2:4])
print(X.size()[1])
print("*"*50)
print(y[0:5])
print(y.size())
print("*"*50)
print(X_dev.size())
print(y_dev.size())

tensor([[ 1.0882,  0.2959,  0.1809,  ...,  0.0173,  0.0190,  0.0142],
        [ 0.2261,  0.2763,  0.2607,  ...,  0.0552,  0.0091, -0.0157]],
       device='cuda:0')
4608
**************************************************
tensor([1, 1, 1, 1, 1], device='cuda:0')
torch.Size([5600])
**************************************************
torch.Size([800, 4608])
torch.Size([800])


In [15]:
# parameters = {
#     'max_depth': [ 6, 7, 9,10],
#     'num_leaves':[ 60, 80, 90, 100],
# }

# gbm = lgb.LGBMClassifier(objective = 'multiclass',
#                           metric = 'multi_logloss',
#                           max_depth = 6,
#                           num_leaves = 60,
#                           learning_rate = 0.1,
#                           feature_fraction = 0.7,
#                           min_child_samples=21,
#                           min_child_weight=0.001,
#                           bagging_fraction = 1,
#                           bagging_freq = 2,
#                           reg_alpha = 0.001,
#                           reg_lambda = 8,
#                           cat_smooth = 0, 
#                           device = 'gpu',
#                           gpu_platform_id = 0, 
#                           gpu_device_id = 0
#                         )
# gsearch = GridSearchCV(gbm, param_grid=parameters, cv=3)
# gsearch.fit(X, y)
# print('参数的最佳取值:{0}'.format(gsearch.best_params_))
# print('最佳模型得分:{0}'.format(gsearch.best_score_))
# print('最佳模型時間:{0}'.format(gsearch.refit_time_))
# print(gsearch.cv_results_['mean_test_score'])
# print(gsearch.cv_results_['params'])

In [ ]:


X = X.cpu()
y = y.cpu()
X_dev = X_dev.cpu()
y_dev = y_dev.cpu()
parameters = {
              # 'device':'gpu',
              'max_depth': [ 25],
              'num_leaves':[110],

              # 'min_data_in_leaf':[20],
              # 'min_sum_hessian_in_leaf':[1e-3],
              
              # 'feature_fraction': [0.6],
              # 'bagging_fraction': [1.0],
              # # # 'bagging_freq': [2, 4, 5, 6, 8],
              # 'lambda_l1': [0.8],
              # 'lambda_l2': [30],

              # 'learning_rate': [0.2],
              # # 'cat_smooth': [10, 15, 20, ]
}

best_lgbm = None
best_acc = 0
f1, f2 = np.meshgrid(parameters['max_depth'],parameters['num_leaves'])
f1 = f1.flat[:]
f2 = f2.flat[:]

for i, j in zip(f1, f2):
  lgbm = lgb.LGBMClassifier(objective = 'muilti_class',
                          metric = 'multi_logloss',
                          max_depth = i,
                          num_leaves = j,
                          learning_rate = 0.1,
                          feature_fraction = 0.7,
                          min_child_samples=21,
                          min_child_weight=0.001,
                          bagging_fraction = 1,
                          bagging_freq = 2,
                          reg_alpha = 0.001,
                          reg_lambda = 8,
                          cat_smooth = 0, 
                          device = 'gpu',
                          gpu_platform_id = 0, 
                          gpu_device_id = 0
                          )
  lgbm.fit(X, y)
  
  y_dev_pred = torch.tensor(lgbm.predict(X_dev))
  acc = sum(y_dev_pred == y_dev).item()/800
  print(f'{i}, {j}, acc in dev:{acc}')
  if best_acc <= acc:
    best_lgbm = lgbm
    best_acc = acc
  
print(best_lgbm, acc) 

[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1


In [ ]:
type(y)

In [ ]:
print(X_dev.size())
print(y_dev.size())

In [ ]:
import seaborn as sn
df_cm = pd.DataFrame(cm.numpy().astype(np.int32),
                     index = [i for i in ["true_A", "true_B", "true_C"]],
                     columns = [i for i in ["pred_A", "pred_B", "pred_C"]])
plt.figure(figsize = (4, 4))
sn.heatmap(df_cm, annot=True, cmap="BuPu")

In [ ]:
cm.numpy().astype(np.int32)

In [ ]:
測試model1的acc

In [ ]:
print(preds1)